# ChatGPT Limits on Code Generation: An Example

### Practical Data Science: 2nd Assignement

#### Dimitris Tsirmpas 

## The Task


You are asked to annotate each proverb, in order to create groups (clusters) of similar proverbs. Simply use the dropdown list in the Excel file to select one cluster ID for each proverb.

1. There’s no requirement to use all the available cluster IDs.
2. There's no single solution for your grouping, but you can keep notes per row (e.g., if a prover belongs to two clusters).
3. You are not allowed to use the web or any other resources (e.g., books).
4. Please provide also a description per cluster ID you used, in the 2nd column of the “cluster labels" tab of the spreadsheet, after you complete the annotation task. For example:
    `cluster id: 0 => corresponds to proverbs that talk about love.`
    `cluster id: 1 => corresponds to proverbs that mention an astronaut.`
5. Submit the excel file, filled.
6. Submit also a text file outlining any difficulties you may have encountered during the task, along with what would you do differently in the next annotation round to ease the work of future annotators.

The annotated dataset can be found in [this file](X2_annotations.xlsx). 

## Notes

### Methodology

The annotation was performed in the following steps:

1. Import the xlsx file into jupyter notebook
2. Assign a relatively specific label to each proverb
3. Use the pandas library to view common labels and organize the dataset
4. Group specific labels to more general labels
5. Automatically assign each distinct label (cluster) to an integer from 1 to 10
6. Export the integer labels to the xlsx file and fill in cluster descriptions


### Difficulties:

- The dataset was extremely uncommon (non common Greek dialects) which meant traditional ML methods for text such as LDA could not be used.
- The proverbs were at times difficult to understand, forcing us to use the "none" label when classification was extermely unreliable
- Some proverbs shared multiple clusters, fuzzy clustering may have been preferred in order to accurately convey their meaning


### Possible future improvements

- Use multi-label annotation (fuzzy clustering) to more accurately convey the proverbs' meaning
- Perhaps use simple NLP algorithms to estimate (even with low confidence) the subject of a proverb, speeding up annotation
- Documenting the decisions we took for hard-to-classify proverbs
- Include pandas code in order for future annotators to be able to easily visualize / manage the dataset

## Step 1: Prompting

We can use a LLM to make initial labels which we can then manually review and group together.

We will use a **2-shot** prompting scheme. The prompt was thus written as follows:

```
You will be given a list of Greek proverbs. Assign each to a category such as "love", "money", "warning against", "work" etc according to it's meaning

Examples:
Input: "Έργατα και δουλείας"
Justification: Contains the words "εργατα" and "δουλειας" which correspond to greek meanings of work
Category: Work

Input: "Σαν θέλ' η νύφ' κι η γαμπρός μπομπή κ' τα συμπεθέρια"
Justification: The input text contains words such as "νυφ'" (νυφη), "γαμπρος", "συμπεθερια" which correspond to marriage
Category: Marriage

Input: "Ο πετεινόν που κουγίζει παρώρας, κόφτει το κεφάλιν ατ"
Justification:
```

The output of the LLM has been pasted below.

In [1]:
import pandas as pd


raw = """Όποιος δεν τον ξέρ΄ ακριβά τον αγοράζει - Money
    Τα 'θελες και τα 'παθες - Fate
    Αδειανός μασκαράς, παραχέρ 'τ' χουριού - Mockery
    Ξεπεσομένος άρχοντας, ζωντανοκολασμένος, λιοντάρι ψόφιο, χαρτί μουρδουλωμένο - Downfall
    Κάλλιο γαιδουρόδερνε, παρά γαιδουρογύρευε - Work ethic
    Εφτό σύρνεται - Patience
    Φύϊτι να μη σας κοιτιλήση του βώδι Αμ' που 'νιν, πρέ του βώδιν; - Procrastination
    Άφσ' τσου πλούσους να σωδιάζουν για τη φτώχεια απανωτιάζουν - Wealth
    Η παλαβάδα 'νι 'κατό λουγιώ στού κόσμου - Eccentricity
    Στού Μπαλίκ Παζάρ ένας παπάς κρέμιτι - Hypocrisy
    Τρεις στα Γέννα, τρεις τα Φώτα τσ' έξη στην Ανάστασι - Religious tradition
    Σαν κρυφτόξυλο δαγκάνει - Secrets
    Σ' ούλα τα καλά, ο Θεός είναι μέσα - Faith
    Ηβάρεσιν, η μοίρα του - Destiny
    Τρέχ' η μύτ' σ' γαμβρε! Τρέχ' απού του χ'μώνα. Σι ξέρου κι απου του καλουκαίρ' - Relationships
    Οποίος προσκυνάει και κλάνει, κ' η πορδαίς του φύγανε, και τση μετάνοιαις του έχασε - Hypocrisy
    Το μήλο θα πέση κάτ' α' τη μηλιά - Inevitability
    Εκαμένμ μ' ασσίμ μουδούριν - Regret
    Καλού καιρού ξύλο - Timing
    Νάχα πουτάνας ριζικό και ά καμάτρας μοίρα - Character
    Άλλος κούκος λαλεί - Individuality
    Έργατα και δουλείας - Work
    Η δουλειά δέ χρίζει μούτρα - Work ethic
    Τσ' η πούλλα τσα πίνει νερό, τοθ Θεό ξαννοίει - Honesty
    Αλλοίς τον οπού καρτερά στα ξένα να δειπνήση και δε(ν) στένει την τσούκαν του, να φάη και να χαρίση - Hospitality
    Παντρεύει Δεσπότη! - Marriage
    Ασκημέ μου στρώσε το έλα κι ομορφέ μου τι θα φάμε - Beauty
    Κάνει τα μπορεί και τα δύνεται - Determination
    Που δεν εντρέπεται, πορεύγεται - Shamelessness
    Ούτε διάκο σε είδαμε, ούτε Πρωτοσύγγελο! Και συ δεσπότης έγεινες! - Surprise
    Τουν έκανα απ 'άσπρου - Innocence
    Λάου λάου πάει - Patience
    Τα δανεικά φορέματα μήτε ζεσταίνουν μήτε ζεσταίνονται - Borrowing
    Βουβός περνά - Silence
    Όπου τσυν' γα δυό λαγούς, χάνει τσαί τον ένα νάνει τσαί τον άλλε - Greed
    Ποντές ξεζεύγουν οι γεωργοί οι κερατάδες - Unity
    Τώρα που γιν η θάλασσα μέλι έσπασε το κουτάλι - Missed opportunities
    Τα θέλ' όλα δικά του - Greed
    Όποιος συχνα σκουντάβγει συχνα και μεταννοιώνει - Carelessness
    Πέσε πίτα να σί φάου - Opportunity
    Της κακομούσουρης το προικιόν η ώμορφη τόχει ριζικό - Appearance vs. Character
    Παρασκεβγοπλυμένη μου και σαββατοστεγνώστρα - Appearances
    Χα θα θέλη η νύφφη κ' η γαμπρός τύφλα να 'χη η πιθθιρός - Love
    Γεια σου γέρω! Κουτσιά σπέρνω - Aging
    Καθένας έχει το δαιμόνιόν του - Individuality
    Των φρενίμων τα παλιά οι λωλοί τα κατελούν - Madness
    Απ' όξω μπέλλα μπέλλα κι από μέσα κατσιβέλλα - Deception
    Καλύτερα προίκα σε κτήματα παρά σε φορέματα - Priorities
    Άρπαξέ με τζαί να σ' αρπάξω - Desire
    Μια φορά πηγαίνει το κολοκύθι για νερό δυό, την τρίτη τσακίζεται - Procrastination
    Κατά τον άνθρωπο και τα ραβδιά - Accountability
    Μόνον εις τους πλουσίους και τους τιμίους δανείζουν! - Class distinction
    Άη μου Νικόλα, και σώσε με. Λέει, Και σάλεβγε κι΄εσύ τα χέρια σου - Superstition
    Θώριε στραβέ. Λέει Θώριε, Θεέ - Divine intervention
    Ο πετεινόν που κουγίζει παρώρας, κόφτει το κεφάλιν ατ - Arrogance
    Ποιός ζει, ποιός πεθαίνει - Mortality
    Κάνω ένανε κοιλό και λαγήνι - Deception
    Μοναχός σου χόρευε κι όσο θέλεις πήδάγε - Independence
    Άνθρωπος ακάλεστος στογ γάμοσ σαν τογ γάδαρον - Marriage
    Ρίξ', ρίξ' γαμπρέ μ' πλατπούτσ' - Marriage
    Όποιος 'ς το κόσμο δε γλεντά και το κρασί δεν πίνει άθρωπος δε λογίζεται όσο χρονώ κι α γίνη - Celebration
    Όποιος πειναει για να πλουτήση, μόν' η πείνα θα του μείνη - Greed
    Άντρα μου, η ξύξες σου τρέχουσι(ν),  Είνε γεναίκα, που το χειμώνα.  Σε ξέρω δα, άντρας μου και που το καλοκαίρι - Gender roles
    Να καναν οι μύγες μέλι τρεις οκάδες στον παρά - Unattainable goals
    Σαν θέλ' η νύφ' κι η γαμπρός μπομπή κ' τα συμπεθέρια - Marriage
    Όποιος 'ς τον κόσμο δε γλεντά και το κρασί δεν πίνει άθρωπος δε λογίζεται όσο χρονώ κι α γίνη - Celebration
    Όποιος πειναει για να πλουτήση, μόν' η πείνα θα του μείνη - Greed
    Άντρα μου, η ξύξες σου τρέχουσι(ν), Είνε γεναίκα, που το χειμώνα. Σε ξέρω δα, άντρας μου και που το καλοκαίρι - Gender roles
    Να καναν οι μύγες μέλι τρεις οκάδες στον παρά - Unattainable goals
    Σαν θέλ' η νύφ' κι η γαμπρός μπομπή κ' τα συμπεθέρια - Marriage
    Όποιος έχει σειέται κι όπου δεν έχει ξυέται - Ambition
    Και όποιος θε να σε κρεμάση ποιός το λε' μη θαν ο ετός σου κι όποιος θε να σε γλυτώση ποιός το λε' μη θαν ο χθρός σου - Fate
    Θέλωντας ο νοικοκύρης και μη θέλωντας ο ζωγράφος βάστα και συ Χριστέ κόκκινα ποδήματα - Envy
    Αν μου πιάσης την μύτη σκάω - Nose
    Ο κινάς κ' η Τρεμιντίνα κάνουν τη γριά καντίνα - Gossip
    Όταν αγιαστούν τα νερά ησυχάζ' ο καιρός - Optimism
    Ευκή ονιού αόραζε και μη βουδί αλοάρι - Honesty
    Πλέει τ' όνομα εις τον νου μου - Reputation
    Η καμήλα εγ κουτσανίσκει που ταφτίν - Rumors
    Αρχίντσεν τα παρδαλά - Madness
    Απόξω κούκλα κι' από μέσα πανούκλα - Deception
    Είχα το παιδί κι είχα τη χαρά κι έψηνα του πέντ' αυγά κι έτρωγα τα τέσσερα και το μισό κι απου το μισό τ' άλλο μισό πόσο 'τρωε το παιδίν; - Parenthood
    Εσηκώθηκ' η Νοθιά με τα παιδογγονά τζη - Gossip
    Βούρλα μού δώνεις, ψαθί σού πλέκω - Sarcasm
    Ξερό ψωμί, καλή καρδιά, φέρνουν ζωή και νιάτα, όχι περόνια ολόχ' σα τσαί ασημένια πιάτα - Values
    Τέτοιου τσιφάλ', τέτοια θέλ' - Preferences
    Τουν γιλάει ου καθρέφτ'ς - Vanity
    Όπου ακούσης μέγα τρύγος, παίρνε μικρό καλάθι - Being cautious
    Σκυλλί γαυγίζει, καρ'βαν πιρνα - False alarms
    Τη δουλειά σου και το γιουμούρ' σου να τους ξενυχτάς - Work ethic
    Κάμνετε πολύ, πες λίγ' - Humility
    Μπισάς κανέναν απ' αυτούς, που δεν τον πρέπεις να μισήσ'ς, απ' αυτούς που δεν τον πρέπεις να μισήσ'ς - Choosing your battles
    Τρέλα η καμήλα που γάλα έχ' πιη, χάνει το βρέφος και το κάμν' μιντρίνα - Addiction
    Στον κρυφό αγάπησεν, ο φανερόν αγαπών να μη πληγωθή - Love
    Τη μπόνα του την άσκηση την έκαμε τζαι μισήσεις την καρκαλιά την έκαμε τζαι μη τ' αγαπήσεις - Exercise
    Όποιος ψωνά, κι' αν τσι ψωνά, και κατούγει - Greed
    Πες με να σ' πω - Rumors
    Έχ' το μπρόστ' σου το μικρό, μην έχ' το μπρόστ' σου τ' άλλο - Self awareness
    Τα πολλά μπόγα στο καμάκι είνε κ' η κρυφή μην είνε μ' - Gluttony
    Πείναει ο άλλος, έτσι του λες; Φονιά ήθικα είνε τα κουμπάκια της - Empathy"""

## Step 2 - Organizing the LLM output
Let's now format our data in a pandas dataframe.

In [2]:
entries = raw.split("\n")
print(len(entries))
print(entries[1:5])

100
["    Τα 'θελες και τα 'παθες - Fate", "    Αδειανός μασκαράς, παραχέρ 'τ' χουριού - Mockery", '    Ξεπεσομένος άρχοντας, ζωντανοκολασμένος, λιοντάρι ψόφιο, χαρτί μουρδουλωμένο - Downfall', '    Κάλλιο γαιδουρόδερνε, παρά γαιδουρογύρευε - Work ethic']


In [3]:
proverbs = []
labels = []

for entry in entries:
    # split data from label on the last '-'
    proverb, label = entry.split('-')
    proverbs.append(proverb)
    labels.append(label)

print(len(proverbs))
print(len(labels))
print(proverbs[1:5])
print(labels[1:5])

100
100
["    Τα 'θελες και τα 'παθες ", "    Αδειανός μασκαράς, παραχέρ 'τ' χουριού ", '    Ξεπεσομένος άρχοντας, ζωντανοκολασμένος, λιοντάρι ψόφιο, χαρτί μουρδουλωμένο ', '    Κάλλιο γαιδουρόδερνε, παρά γαιδουρογύρευε ']
[' Fate', ' Mockery', ' Downfall', ' Work ethic']


In [4]:
df = pd.DataFrame({"proverb": proverbs, "category": labels})
df

,proverb,category
0,Όποιος δεν τον ξέρ΄ ακριβά τον αγοράζει,Money
1,Τα 'θελες και τα 'παθες,Fate
2,"Αδειανός μασκαράς, παραχέρ 'τ' χουριού",Mockery
3,"Ξεπεσομένος άρχοντας, ζωντανοκολασμένος, λ...",Downfall
4,"Κάλλιο γαιδουρόδερνε, παρά γαιδουρογύρευε",Work ethic
...,...,...
95,"Όποιος ψωνά, κι' αν τσι ψωνά, και κατούγει",Greed
96,Πες με να σ' πω,Rumors
97,"Έχ' το μπρόστ' σου το μικρό, μην έχ' το μπ...",Self awareness
98,Τα πολλά μπόγα στο καμάκι είνε κ' η κρυφή ...,Gluttony


And ensure there are no trailing spaces in our categories, since we will need to frequently select them.

In [5]:
strip_func = lambda x: x.strip()
df.proverb = df.proverb.apply(strip_func)
df.category = df.category.apply(strip_func)

df

,proverb,category
0,Όποιος δεν τον ξέρ΄ ακριβά τον αγοράζει,Money
1,Τα 'θελες και τα 'παθες,Fate
2,"Αδειανός μασκαράς, παραχέρ 'τ' χουριού",Mockery
3,"Ξεπεσομένος άρχοντας, ζωντανοκολασμένος, λιοντ...",Downfall
4,"Κάλλιο γαιδουρόδερνε, παρά γαιδουρογύρευε",Work ethic
...,...,...
95,"Όποιος ψωνά, κι' αν τσι ψωνά, και κατούγει",Greed
96,Πες με να σ' πω,Rumors
97,"Έχ' το μπρόστ' σου το μικρό, μην έχ' το μπρόστ...",Self awareness
98,Τα πολλά μπόγα στο καμάκι είνε κ' η κρυφή μην ...,Gluttony


And now let's see how many and which labels the LLM chose to select.

In [6]:
print(len(set(df.category)))
sorted(set(df.category))

75


['Accountability',
 'Addiction',
 'Aging',
 'Ambition',
 'Appearance vs. Character',
 'Appearances',
 'Arrogance',
 'Beauty',
 'Being cautious',
 'Borrowing',
 'Carelessness',
 'Celebration',
 'Character',
 'Choosing your battles',
 'Class distinction',
 'Deception',
 'Desire',
 'Destiny',
 'Determination',
 'Divine intervention',
 'Downfall',
 'Eccentricity',
 'Empathy',
 'Envy',
 'Exercise',
 'Faith',
 'False alarms',
 'Fate',
 'Gender roles',
 'Gluttony',
 'Gossip',
 'Greed',
 'Honesty',
 'Hospitality',
 'Humility',
 'Hypocrisy',
 'Independence',
 'Individuality',
 'Inevitability',
 'Innocence',
 'Love',
 'Madness',
 'Marriage',
 'Missed opportunities',
 'Mockery',
 'Money',
 'Mortality',
 'Nose',
 'Opportunity',
 'Optimism',
 'Parenthood',
 'Patience',
 'Preferences',
 'Priorities',
 'Procrastination',
 'Regret',
 'Relationships',
 'Religious tradition',
 'Reputation',
 'Rumors',
 'Sarcasm',
 'Secrets',
 'Self awareness',
 'Shamelessness',
 'Silence',
 'Superstition',
 'Surprise',


## Step 3 - Grouping the LLM output

We will now manually group the LLM labels into our own clusters:

In [7]:
cat_dict = {
    'Accountability': "virtue",
    'Addiction': "sin",
    'Aging': "life",
    'Ambition': "virtue",
    'Appearance vs. Character': "life",
    'Appearances': "life",
    'Arrogance': "sin",
    'Beauty': "virtue",
    'Being cautious': "life",
    'Borrowing': "sin",
    'Carelessness': "sin",
    'Celebration': "life",
    'Character': "none",
    'Choosing your battles': "none",
    'Class distinction': "money",
    'Deception': "sin",
    'Desire': "sin",
    'Destiny': "life",
    'Determination': "virtue",
    'Divine intervention': "religion",
    'Downfall': "life",
    'Eccentricity': "sin",
    'Empathy': "virtue",
    'Envy': "sin",
    'Exercise': "none",
    'Faith': "religion",
    'False alarms': "sin",
    'Fate': "religion",
    'Gender roles': "life",
    'Gluttony': "sin",
    'Gossip': "sin",
    'Greed': "sin",
    'Honesty': "virtue",
    'Hospitality': "virtue",
    'Humility': "virtue",
    "Hypocrisy": "sin",
    "Independence": "virtue",
    "Individuality": "virtue",
    "Inevitability": "life",
    "Innocence": "virtue",
    "Madness": "none", #??
    "Marriage": "marriage",
    "Missed opportunities": "life",
    "Mockery": "sin",
    "Money": "money",
    "Mortality": "life",
    "Nose": "life",
    "Opportunity": "life",
    "Optimism": "virtue",
    "Love": "love",
    "Parenthood": "marriage",
    "Patience": "virtue",
    "Preferences": "life",
    "Priorities": "life",
    "Procrastination": "sin",
    "Regret": "conflict",
    "Relationships": "marriage",
    "Religious tradition": "religion",
    "Reputation": "virtue",
    "Rumors": "conflict",
    "Sarcasm": "conflict",
    "Secrets": "sin",
    "Self awareness": "virtue",
    "Shamelessness": "sin",
    "Silence": "life", #??
    "Superstition": "religion",
    "Surprise": "none",
    "Timing": "conflict",
    "Unattainable goals": "money",
    "Unity": "virtue",
    "Values": "virtue",
    "Vanity": "sin",
    "Wealth": "money",
    "Work": "work",
    "Work ethic": "work",
}

In [8]:
grouped_df = df.copy()
grouped_df.category = df.category.apply(lambda x : cat_dict.get(x) if cat_dict.get(x) != None else x)
grouped_df

,proverb,category
0,Όποιος δεν τον ξέρ΄ ακριβά τον αγοράζει,money
1,Τα 'θελες και τα 'παθες,religion
2,"Αδειανός μασκαράς, παραχέρ 'τ' χουριού",sin
3,"Ξεπεσομένος άρχοντας, ζωντανοκολασμένος, λιοντ...",life
4,"Κάλλιο γαιδουρόδερνε, παρά γαιδουρογύρευε",work
...,...,...
95,"Όποιος ψωνά, κι' αν τσι ψωνά, και κατούγει",sin
96,Πες με να σ' πω,conflict
97,"Έχ' το μπρόστ' σου το μικρό, μην έχ' το μπρόστ...",virtue
98,Τα πολλά μπόγα στο καμάκι είνε κ' η κρυφή μην ...,sin


Let's make sure we didn't miss any LLM labels.

In [9]:
set(grouped_df.category.values)

{'conflict',
 'life',
 'love',
 'marriage',
 'money',
 'none',
 'religion',
 'sin',
 'virtue',
 'work'}

In order to export the clusters into the excel file, we need to assign unique cluster IDs to them. This can be achieved easily using pandas:

In [10]:
# begin numbering from 1
d = {ni: indi+1 for indi, ni in enumerate(set(grouped_df.category.values))}
d

{'work': 1,
 'conflict': 2,
 'sin': 3,
 'virtue': 4,
 'marriage': 5,
 'love': 6,
 'religion': 7,
 'money': 8,
 'life': 9,
 'none': 10}

In [11]:
numbers = [d[ni] for ni in grouped_df.category.values]
print(grouped_df.category[1:5])
print(numbers[1:5])

1    religion
2         sin
3        life
4        work
Name: category, dtype: object
[7, 3, 9, 1]


In [12]:
grouped_df = grouped_df.assign(code=numbers)
grouped_df

,proverb,category,code
0,Όποιος δεν τον ξέρ΄ ακριβά τον αγοράζει,money,8
1,Τα 'θελες και τα 'παθες,religion,7
2,"Αδειανός μασκαράς, παραχέρ 'τ' χουριού",sin,3
3,"Ξεπεσομένος άρχοντας, ζωντανοκολασμένος, λιοντ...",life,9
4,"Κάλλιο γαιδουρόδερνε, παρά γαιδουρογύρευε",work,1
...,...,...,...
95,"Όποιος ψωνά, κι' αν τσι ψωνά, και κατούγει",sin,3
96,Πες με να σ' πω,conflict,2
97,"Έχ' το μπρόστ' σου το μικρό, μην έχ' το μπρόστ...",virtue,4
98,Τα πολλά μπόγα στο καμάκι είνε κ' η κρυφή μην ...,sin,3


And finally, we can take the string of all cluster IDs and paste them into the excel file.

In [13]:
",".join(grouped_df.code.apply(str))

'8,7,3,9,1,4,3,8,3,3,7,3,7,9,5,3,9,2,2,10,4,1,1,4,4,5,4,4,3,10,4,4,3,9,3,4,9,3,3,9,9,9,6,9,4,10,3,9,3,3,4,8,7,7,3,9,3,4,5,5,9,3,9,8,5,9,3,9,8,5,4,7,3,9,3,4,4,4,2,10,3,5,3,2,4,9,3,9,3,1,4,10,3,6,10,3,2,4,3,4'